In [202]:
import pandas as pd
import altair as alt
import matplotlib.colors as mcolors

In [203]:
#Color Schemes for Altair Visualizations

# Updated Base Colors for Contrast on Dark Gray Background
base_colors = [
    "#23479C",  # darkest blue
    "#2063AA",
    "#2282B8",
    "#2D9EC0",
    "#45B4C2",
    "#69C5BD",
    "#94D5B9",
    "#BDE5B5",
    "#DBF1B4"   # lightest yellow
]
base_colors_darker = [
    "#23479C",  # darkest blue
    "#2063AA",
    "#2282B8",
    "#288BA9",
    "#3495A2",
    "#3C9A92",
    "#3D9E74",
    "#459735",
    "#73A41E"   # lightest yellow
]

base_colors_4 = [
    "#23479C",  # darkest blue
    "#2282B8",
    "#3C9A92",
    "#73A41E"   # lightest yellow
]



# --- Color Schemes ---

## Categorical: Directly using refined base colors
categorical_colormap = mcolors.ListedColormap(base_colors)

## Sequential: Create a smooth gradient across the full base colors
sequential_colormap = mcolors.LinearSegmentedColormap.from_list(
    'sequential', base_colors, N=256
)
sequential_colors = [sequential_colormap(i/255) for i in range(256)]

## Diverging: Transition adjusted for better visibility
diverging_colormap = mcolors.LinearSegmentedColormap.from_list(
    'diverging', ['#3399ff', '#ffd700'], N=256
)
diverging_colors = [diverging_colormap(i/255) for i in range(256)]


In [204]:
# Data with detailed information (researched and added)
data = pd.DataFrame({
    'Event': [
        'NCAA Prohibition on NIL',
        'O\'Bannon v. NCAA',
        'Alston Decision',
        'NIL Policy Launch',
        'First Major NIL Deals',
        'NIL Collectives Formation',
        'Significant Female Athlete Deal'
    ],
    'Date': [
        '1951-06-01', '2014-08-01', '2021-06-01',
        '2021-07-01', '2021-09-01', '2022-03-01', '2023-05-01'
    ],
    'Category': [
        'Regulation', 'Legal Case', 'Legal Case',
        'Policy', 'Policy', 'Industry Formation', 'Policy'
    ],
    'Description': [
        'Prevented student-athletes from earning any money related to their name, image, or likeness. This was rooted in the idea of preserving amateurism in college sports, but it created a system where athletes brought in millions for their school & the NCAA while receiving no personal profit.',
        'Former UCLA basketball player Ed O’Bannon challenged the NCAA for using his image in video games without permission or payment. The court ruled that the NCAAs restrictions on athlete compensation violated antitrust laws. This case showed that the NCAA could not use amateurism as a blanket defense against legal scrutiny.',
        'Supreme Court ruling that the NCAA could not limit education-related benefits, like scholarships for graduate school or computers, for student-athletes. While the case didn’t directly address NIL, the unanimous ruling made it clear that the NCAA’s compensation restrictions were subject to antitrust law.',
        'Marked the official end of the ban on athletes profiting from their personal brand. For the first time, student-athletes could legally earn money through sponsorships, endorsements, social media content, merchandise, and other avenues.',
        'When the NIL policy went into effect, several athletes immediately signed high-profile deals with brands, local businesses, and national sponsors. The visibility of these early deals helped drive wider participation and encouraged both athletes and brands to explore the new opportunities.',
        'NIL collectives began forming as groups, often backed by donors or alumni, to help athletes at specific schools secure NIL deals. While technically independent from athletic departments, these collectives were often closely linked to school communities and played a major role in recruiting and retaining athletes.',
        'Some of the most notable NIL deals have involved female athletes, especially those with strong social media followings or Olympic-level visibility. These successes highlighted the growing market for women’s sports and helped advance the conversation around equity and representation in college athletics.'
    ]
})

In [205]:

# Convert 'Date' column to datetime and extract 'Year' and 'MonthName'
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['MonthName'] = data['Date'].dt.strftime('%b')

# Define the order of months for sorting
month_order = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
]

# Create a selection object for selecting a year with a custom label name
year_selection = alt.selection_point(
    fields=['Year'],
    bind=alt.binding_select(options=sorted(data['Year'].unique()), name=" Year: "),
    name="Select Year"
)

# Create a multi-selection for category
category_selection = alt.selection_point(fields=['Category'], bind='legend')

# Set consistent height and width for both charts
chart_height = 100
chart_width = 600

# Full timeline chart
full_timeline_chart = alt.Chart(data).mark_point(filled=True, size=50).encode(
    x=alt.X(
        'Year:Q',
        axis=alt.Axis(title='Year', format='.0f'),
        scale=alt.Scale(domain=[1950, 2035])
    ),
    y=alt.Y('Event:N', axis=None),
    color=alt.Color(
        'Category:N',
        legend=alt.Legend(title="Category"),
        scale=alt.Scale(range=base_colors_4) 
    ),
    opacity=alt.condition(category_selection, alt.value(1), alt.value(0.2)),
    tooltip=['Event', 'Date', 'Description']
).properties(
    width=chart_width,
    height=chart_height
).add_params(
    category_selection
)

# Add text labels to the full timeline
full_text = full_timeline_chart.mark_text(
    angle=315,  # Tilt text labels slightly upwards
    align='left',
    baseline='middle',
    dx=7,
    fontSize= 14,
).encode(
    text='Event'
)

# Create a DataFrame to ensure there are always 12 points for each month
months = pd.DataFrame({
    'MonthName': month_order
})

# Base chart for all months
month_base = alt.Chart(months).mark_circle(size=100, opacity=0).encode(
    x=alt.X('MonthName:N', axis=alt.Axis(title='Month', labelAngle=0), sort=month_order)
)

# Zoomed-in chart for the selected year with full months displayed
zoomed_points = alt.Chart(data).mark_point(filled=True, size=100).encode(
    x=alt.X('MonthName:N', axis=alt.Axis(title='Month'), sort=month_order),
    y=alt.Y('Event:N', axis=None),
    color=alt.Color('Category:N', scale=alt.Scale(range=base_colors_4)),
    opacity=alt.condition(category_selection, alt.value(1), alt.value(0.2)),
    tooltip=['Event', 'Date', 'Description']
).transform_filter(year_selection).properties(
    width=chart_width,
    height=chart_height
)

# Add text labels to the zoomed-in chart
zoomed_text = zoomed_points.mark_text(
    angle=315,  # Tilt text labels slightly upwards
    align='left',
    baseline='middle',
    dx=7,
    fontSize=14,
).encode(
    text='Event'
)

# Combine the base chart with zoomed points and text
zoomed_chart = month_base + zoomed_points + zoomed_text

# Combine the full timeline and the filtered zoomed-in chart
combined_chart = (full_timeline_chart + full_text) & (zoomed_chart).add_params(year_selection)

# Configure and display the final combined chart
final_chart = combined_chart.properties(
    title='Timeline of NIL Policy Evolution and Impact'
).configure_axis(
    grid=True,
    gridOpacity=1,
    gridWidth=1,
).configure_legend(
).configure_title(
).configure_view(
    strokeOpacity=0 
).properties(
    background= "#ffffff",  
)

# Display the chart
final_chart


alt.VConcatChart(...)

In [206]:
# Save the interactive Altair chart as an HTML file
final_chart.save('visualizations/1_policy_timeline.html')

In [210]:
import pandas as pd
import altair as alt

# Load data
nil_df = pd.read_csv('projected_nil_funding_by_school.csv')
football_df = pd.read_csv('football.csv')

# Rename columns and merge DataFrames
football_df = football_df.rename(columns={'Team': 'University'})
merged_df = pd.merge(nil_df, football_df, on='University', how='inner')

# Convert 'Win %' to a numerical format for plotting
merged_df['Win %'] = merged_df['Win %'].str.rstrip('%').astype('float') / 100

# Define an interval selection for the scatter plot
interval_selector = alt.selection_interval(encodings=['x', 'y'])

# Define a point selection with multiple selection capability using click (no toggle needed for multiple selects directly)
conference_selector = alt.selection_point(fields=['2024 Conf'], bind='legend', toggle='event.shiftKey')

# Using the base_colors_4 as conference colors
base_colors_4 = [
    "#23479C",  # darkest blue
    "#2282B8",
    "#3C9A92",
    "#73A41E"   # lightest yellow
]
color_scheme = alt.Scale(domain=merged_df['2024 Conf'].unique(), range=base_colors_4)

# Base scatter plot of Win % vs Collective Funding colored by Conference
scatter = alt.Chart(merged_df).mark_circle(size=100).encode(
    x=alt.X('Collective Funding', title='NIL Collective Funding'),
    y=alt.Y('Win %', title='Win Percentage'),
    color=alt.Color('2024 Conf:N', scale=color_scheme, legend=alt.Legend(title='Conference')),
    opacity=alt.condition(conference_selector & interval_selector, alt.value(1), alt.value(0.2)),
    tooltip=['University', 'Collective Funding', 'Win %', '2024 Conf']
).add_params(
    interval_selector,
    conference_selector
).properties(
    width=700,
    height=400
)

# Add a light gray regression line to show correlation
regression_line = alt.Chart(merged_df).transform_regression(
    'Collective Funding', 'Win %'
).mark_line(color='lightgray').encode(
    x='Collective Funding',
    y='Win %'
).transform_filter(
    conference_selector
)

# Add Points along the regression line to show tooltips
regression_points = alt.Chart(merged_df).transform_regression(
    'Collective Funding',
    'Win %',
    extent=[merged_df['Collective Funding'].min(), merged_df['Collective Funding'].max()]
).mark_circle(opacity=0.0, size=100).encode(
    x='Collective Funding',
    y='Win %',
    tooltip=[alt.Tooltip('Collective Funding', title='NIL Collective Funding'), alt.Tooltip('Win %', title='Win Percentage')]
).transform_filter(
    conference_selector
)

# Combine scatter plot with regression line and tooltips
scatter_with_regression_tooltips = scatter + regression_line + regression_points

# Horizontal bar chart of MOV for teams within the selected area in the scatter plot
bars = alt.Chart(merged_df).mark_bar().encode(
    y=alt.Y('University', sort='-x'),
    x=alt.X('MOV', title='Margin of Victory'),
    color=alt.condition(conference_selector & interval_selector, '2024 Conf:N', alt.value('lightgray'), scale=color_scheme),
    tooltip=['University', 'MOV']
).transform_filter(
    interval_selector
).transform_filter(
    conference_selector
).properties(
    width=700,
    height=400
)

# Combine the scatter plot with regression and the horizontal bar chart
chart = scatter_with_regression_tooltips & bars

# Display the chart
chart

alt.VConcatChart(...)

In [211]:
chart.save('visualizations/2_NIL_win.html')